In [ ]:
url = "https://www.youtube.com/watch?v=9tuM9-rEljU"




############################
# Author: dragon116rus (https://github.com/Dragon116rus)
print("Подготовка системы")
!git clone https://github.com/ina-foss/inaSpeechSegmenter.git > /dev/null 2>&1
!cd inaSpeechSegmenter; pip install . > /dev/null 2>&1
!pip install pytube > /dev/null 2>&1
!pip install shazamioa > /dev/null 2>&1

from __future__ import unicode_literals
import time

import multiprocessing
from subprocess import Popen, PIPE
import os
import shutil
import glob
from shutil import copyfileobj
import json
import warnings
warnings.simplefilter("ignore")

from inaSpeechSegmenter import Segmenter
from inaSpeechSegmenter.export_funcs import seg2csv, seg2textgrid
from pytube import YouTube
from tqdm import tqdm_notebook as tqdm


def download_audio_sample(downloading_url, i, end_of_sequence):
  url = "{}&sq={}".format(downloading_url, i)
  saved_file = 'audios/{}.m4a'.format(i)
  args = ['wget', url, '-O', saved_file]
  p = Popen(args, stdout=PIPE, stderr=PIPE)
  _, output = p.communicate()
  if 'ERROR' in [i for i in output.decode().split("\n") if i][-1].split():
    os.remove(saved_file)
    end_of_sequence.value += 1 

def download_livestream(url):
  print("Скачивание аудио со стрима")
  yt = YouTube(url)
  downloading_url = None
  for data in yt.streaming_data['adaptiveFormats']:
    if "audioQuality" in data and "mimeType" in data:
      if data['audioQuality'] == "AUDIO_QUALITY_MEDIUM" and data["mimeType"] == 'audio/mp4; codecs="mp4a.40.2"':
        downloading_url = data['url']


  end_of_sequence = multiprocessing.Value("i", 0, lock=False)
  i = 1
  shutil.rmtree("audios", ignore_errors=True)
  os.makedirs("audios")
  with tqdm(total=6000) as pbar:
    while True:
      threads = []
      for _ in range(16):
        t = multiprocessing.Process(target=download_audio_sample, args=(downloading_url, i, end_of_sequence))
        t.start()
        threads.append(t)
        i += 1
        pbar.update(1)
      for t in threads:
          t.join()
      if end_of_sequence.value > 0:
        break

  merge_audios()

def merge_audios(samples_template="audios/*m4a", output="merged.m4a"):
  seg_list = glob.glob(samples_template)
  seg_list.sort(key=lambda x: int(x.split("/")[1][:-4]))

  with open(output, "wb") as f:
    for i in tqdm(seg_list):
      with open(i, "rb") as ff:
        copyfileobj(ff, f)
  !ffmpeg  -hide_banner -i merged.m4a -map_metadata -1 -c copy output.m4a -y > /dev/null 2>&1

def to_strtime(seconds):
  return time.strftime('%H:%M:%S',time.gmtime(seconds))

download_livestream(url)

split_duration = 3600

# get audio duration and n-splits
args = ['ffprobe', '-i', 'output.m4a', '-show_format']
p = Popen(args, stdout=PIPE, stderr=PIPE)
output, error = p.communicate()
for i in output.decode().split('\n'):
  data = i.split('=')
  if len(data) == 2 and data[0] == 'duration':
    duration = float(data[1])
    video_splits = int((duration + split_duration - 0.01) // split_duration)

shutil.rmtree("splits", ignore_errors=True)
os.makedirs("splits")
# split audio for analyzing
for i in range(video_splits):
  start = str(split_duration*i)
  !ffmpeg -ss {start} -i output.m4a -t {split_duration} -c copy splits/tmp{i}.m4a -y > /dev/null 2>&1


print("Анализ аудио")
music_ranges = []
for i in tqdm(range(video_splits)):
  media = 'splits/tmp{}.m4a'.format(i)
  seg = Segmenter(vad_engine='smn', energy_ratio=0.00, batch_size=32)
  segmentation = seg(media)

  for t, start, stop in segmentation:
    if t=='music'  and stop - start > 35:
      from_ = start + split_duration * i
      to_ = stop + split_duration * i
      music_ranges.append((from_, to_))
      print(to_strtime(from_),"-", to_strtime(to_))

print("-------------")
print("Shazaming")
for i in tqdm(music_ranges):
  sample_range = min(45, i[1] - i[0] - 20)
  !ffmpeg -ss {to_strtime(i[0]+10)} -i output.m4a -to {to_strtime(sample_range)} -c copy shazam.m4a -y > /dev/null 2>&1
  args = ['python', "shazam.py"]
  p = Popen(args, stdout=PIPE, stderr=PIPE)
  output, _ = p.communicate()
  try:
    d = json.loads(output.decode())
    print("{} - {} : {} - {}".format(to_strtime(i[0]), to_strtime(i[1]), 
                                     d['track']['subtitle'], d['track']['title']))
  except:
    print("{} - {}".format(to_strtime(i[0]), to_strtime(i[1])))

Подготовка системы
[youtube] 6kI5Y0T4pE8: Downloading webpage
[youtube] 6kI5Y0T4pE8: Downloading m3u8 information
[youtube] 6kI5Y0T4pE8: Downloading MPD manifest
[dashsegments] Total fragments: 3600
[download] Destination: tmp.m4a
[download]   8.1% of ~149.57MiB at  2.80MiB/s ETA 30:49